# Connecting to Drive

In [2]:
# Connecting to google drive
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Path
%cd "/content/drive/MyDrive/Bridge construction health/brridge-failure-detection"

/content/drive/MyDrive/Bridge construction health/brridge-failure-detection


In [26]:
! ls -ltrh

total 4.6M
-rw------- 1 root root 868K Aug 19 15:51 data.rar
-rw------- 1 root root 3.4M Aug 19 15:51 data1.rar
-rw------- 1 root root 373K Aug 21 16:26 wavelet.ipynb
-rw------- 1 root root 3.0K Aug 21 16:26 sst_nino3.dat
-rw------- 1 root root  35K Aug 21 16:26 analysis.ipynb
drwx------ 5 root root 4.0K Aug 21 16:35 data
drwx------ 3 root root 4.0K Aug 21 16:36 data1
-rw------- 1 root root 7.8K Aug 21 16:55 github_handler.ipynb
-rw------- 1 root root 4.9K Aug 21 16:57 bridge_failure_detection.ipynb


# Imports and installations

In [25]:
import os
import shutil
from pathlib import Path

In [31]:
from PIL import Image
import numpy as np

# Unzipping the dataset of CWT images

In [ ]:
! unrar x data.rar ./

In [ ]:
! unrar x data1.rar ./data1

# Merging datasets

In [28]:
def merge_directories(dir1, dir2, output_dir):
    # Convert paths to Path objects
    dir1 = Path(dir1)
    dir2 = Path(dir2)
    output_dir = Path(output_dir)

    # Create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)

    # Get a list of all subdirectories in both input directories
    subdirs = set(os.listdir(dir1)).union(set(os.listdir(dir2)))

    # Iterate over each subdirectory
    for subdir in subdirs:
        src_subdir1 = dir1 / subdir
        src_subdir2 = dir2 / subdir
        dst_subdir = output_dir / subdir

        # If the subdir exists in the first source directory, copy it to the output
        if src_subdir1.exists():
            shutil.copytree(src_subdir1, dst_subdir, dirs_exist_ok=True)

        # If the subdir exists in the second source directory, merge it with the output
        if src_subdir2.exists():
            for item in src_subdir2.iterdir():
                src_item = src_subdir2 / item.name
                dst_item = dst_subdir / item.name

                if src_item.is_dir():
                    # If it's a directory, copy it recursively
                    shutil.copytree(src_item, dst_item, dirs_exist_ok=True)
                else:
                    # If it's a file, copy it, overwriting if necessary
                    shutil.copy2(src_item, dst_item)

In [29]:
dir1 = "./data"
dir2 = "./data1/data1"
output_dir = "./main_datatset"

merge_directories(dir1, dir2, output_dir)

# Vectorization of images

In [32]:
def vectorize_rgb_image_to_array(png_path):
    # Load the PNG image
    image = Image.open(png_path)

    # Convert the image to RGB mode (if it's not already)
    image = image.convert('RGB')

    # Convert the image to a numpy array
    img_array = np.array(image)

    return img_array

In [34]:
! ls "./main_datatset/After_reconstruction" | head

1_After_reconstruction_0_1.png
1_After_reconstruction_0_2.png
1_After_reconstruction_0_3.png
1_After_reconstruction_0_4.png
1_After_reconstruction_0_5.png
1_After_reconstruction_100_1.png
1_After_reconstruction_100_2.png
1_After_reconstruction_100_3.png
1_After_reconstruction_100_4.png
1_After_reconstruction_100_5.png


## Checking the sample number of each class

In [39]:
len(os.listdir("./main_datatset/When_rebuilding"))

1037

In [37]:
len(os.listdir("./main_datatset/Before_renovation"))

1033

In [36]:
len(os.listdir("./main_datatset/After_reconstruction"))

1024

## Vetorizing all

In [50]:
X = []
y = []

In [46]:
all_classes = os.listdir("./main_datatset/")

In [48]:
all_classes_dict = {
    "Before_renovation": 0,
    "After_reconstruction": 1,
    "When_rebuilding": 2
}

In [51]:
# Vectorizing and added labels of images
for lbl_class in all_classes:
  print(lbl_class)
  images = os.listdir(f"./main_datatset/{lbl_class}")

  for img in images:
    img_path = f"./main_datatset/{lbl_class}/{img}"
    img_array = vectorize_rgb_image_to_array(img_path)
    X.append(img_array)
    y.append(all_classes_dict[lbl_class])

Before_renovation
When_rebuilding
After_reconstruction


# Model Architecture

In [65]:
from sklearn.model_selection import train_test_split

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [66]:
X_train.shape

(2475, 23, 23, 3)

In [73]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(23, 23, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 21, 21, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 8, 8, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,187 (332.76 KB)

 Trainable params: 85,187 (332.76 KB)

 Non-trainable params: 0 (0.00 B)

In [74]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.3730 - loss: 7.1989 - val_accuracy: 0.4992 - val_loss: 1.0140
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5333 - loss: 0.9906 - val_accuracy: 0.4297 - val_loss: 1.1686
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.5213 - loss: 0.9686 - val_accuracy: 0.5638 - val_loss: 0.9642
Epoch 4/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.6242 - loss: 0.8074 - val_accuracy: 0.6494 - val_loss: 0.8337
Epoch 5/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6443 - loss: 0.7778 - val_accuracy: 0.5961 - val_loss: 0.8125
Epoch 6/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6922 - loss: 0.6864 - val_accuracy: 0.6527 - val_loss: 0.8299
Epoch 7/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7160 - loss: 0.6769 - val_accuracy: 0.6446 - val_loss: 0.7616
Epoch 8/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7046 - loss: 0.6581 - val_accuracy: 0.6769 - v

In [75]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7349 - loss: 1.5217
Test loss: 1.538961410522461
Test accuracy: 0.7205169796943665
